In [1]:
#  Use the appropriate install depending on your local directory

##%pip install --quiet -U pip -r ../requirements/requirements-train.txt ../.

#%pip install --quiet -U pip -r ./requirements/requirements-train.txt ./.

In [2]:
import pickle
import jax
from hydra import compose, initialize

#export PATH="~/Documents/InstadeepGroupProject/jumanji_routing"
from jumanji.training.setup_train import setup_agent, setup_env
from jumanji.training.utils import first_from_device

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


In [3]:
import numpy as np
import chex
import jax.numpy as jnp
import jumanji
from typing import Dict

from jumanji.environments import Connector
from jumanji.environments.routing.connector.types import Agent, Observation, State
from jumanji.types import TimeStep
from jumanji.environments.routing.connector.utils import get_position, get_target
from ic_routing_board_generation.board_generator.jax_board_generation.board_generator_random_seed_rb import RandomSeedBoard
from ic_routing_board_generation.board_generator.numpy_utils.utils import get_heads_and_targets
from ic_routing_board_generation.interface.board_generator_interface_numpy import BoardGenerator, BoardName
from datetime import datetime
from copy import deepcopy

In [4]:
from ic_routing_board_generation.board_generator.numpy_utils.post_processor_utils_numpy import \
    training_board_from_solved_board, extend_wires, count_detours
from ic_routing_board_generation.board_generator.jax_utils.post_processor_utils_jax import \
    training_board_from_solved_board_jax, extend_wires_jax

## Load configs

In [5]:
#with initialize(version_base=None, config_path="../jumanji/training/configs"):
    #cfg = compose(config_name="config.yaml", overrides=["env=connector", "agent=a2c"])
with initialize(version_base=None, config_path="./jumanji/training/configs"):
    cfg = compose(config_name="config.yaml", overrides=["env=connector", "agent=a2c"])
cfg

{'agent': 'a2c', 'seed': 0, 'logger': {'type': 'terminal', 'save_checkpoint': False, 'name': '${agent}_${env.name}'}, 'env': {'name': 'connector', 'registered_version': 'Connector-v0', 'network': {'transformer_num_blocks': 4, 'transformer_num_heads': 8, 'transformer_key_size': 16, 'transformer_mlp_units': [512], 'conv_n_channels': 32}, 'training': {'num_epochs': 500, 'num_learner_steps_per_epoch': 100, 'n_steps': 20, 'total_batch_size': 128}, 'evaluation': {'eval_total_batch_size': 5000, 'greedy_eval_total_batch_size': 5000}, 'a2c': {'normalize_advantage': False, 'discount_factor': 0.99, 'bootstrapping_factor': 0.95, 'l_pg': 1.0, 'l_td': 1.0, 'l_en': 0.01, 'learning_rate': 0.0002}}}

## Load a saved checkpoint

In [6]:
file = "examples/trained_agent_10x10_5_uniform/19-27-36/training_state_10x10_5_uniform"

with open(file,"rb") as f:
    training_state = pickle.load(f)

params = first_from_device(training_state.params_state.params)
#print(params)
env = setup_env(cfg).unwrapped
#print(env)
agent = setup_agent(cfg, env)
#print(agent)
policy = jax.jit(agent.make_policy(params.actor, stochastic = False))
#print(params.num_agents)
#print(policy)


# Roll out a few episodes of their default "uniform" board generation

In [7]:
env_fn = jax.jit(env.reset) # Speed up reset
step_fn = jax.jit(env.step)  # Speed up env.step
GRID_SIZE = 10
NUM_AGENTS = 5
NUM_EPISODES = 10 # 10000

print(datetime.now())
states = []
key = jax.random.PRNGKey(cfg.seed)

connections = []
for episode in range(NUM_EPISODES):  
    key, reset_key = jax.random.split(key)
    #state, timestep = jax.jit(env.reset)(reset_key)
    state, timestep = env_fn(reset_key)
        
        
    while not timestep.last():
        key, action_key = jax.random.split(key)
        observation = jax.tree_util.tree_map(lambda x: x[None], timestep.observation)
        # Two implementations for calling the policy, about equivalent speed
        #action, _ = policy(observation, action_key)
        action, _ = jax.jit(policy)(observation, action_key)
        # Three implementations for updating the state/timestep.  The third is much faster.
        #state, timestep = jax.jit(env.step)(state, action.squeeze(axis=0)) # original jit = 0.32, 52sec/10
        #state, timestep = env.step(state, action.squeeze(axis=0)) # no jit = 0.13, 26sec/10
        state, timestep = step_fn(state, action.squeeze(axis=0)) # jit function = 0.003 5 sec/10, 49sec/100d
        states.append(state)

        
        
    # Freeze the terminal frame to pause the GIF.
    for _ in range(10):
        states.append(state)
        
    #################################################
    # Evaluate the number of wires connected
    num_connected = timestep.extras["num_connections"]
    connections.append(num_connected)

#print(connections)
connections = np.array (connections)
print(f"{NUM_EPISODES} Evaluations: Average connections for default Uniform board = {connections.mean()}, std={connections.std()}")
print(datetime.now())

2023-04-27 01:37:47.073828
10 Evaluations: Average connections for default Uniform board = 3.8, std=0.8717797887081347
2023-04-27 01:37:49.303075


TIME ESTIMATES FOR THEIR UNIFORM GENERATOR

10 BOARDS = 1.2 sec
2023-04-25 14:43:46.898485
10 Evaluations: Average connections for default Uniform board = 3.8, std=0.8717797887081347
2023-04-25 14:43:48.087526


100 BOARDS = 4 SEC
2023-04-25 14:42:45.907124
100 Evaluations: Average connections for default Uniform board = 4.14, std=0.8834025130143109
2023-04-25 14:42:49.580220

1000 BOARDS = 41.4 sec/30.5 SEC
2023-04-25 14:44:18.690546
1000 Evaluations: Average connections for default Uniform board = 4.048, std=0.8908961780140264
2023-04-25 14:44:50.098781

2023-04-25 20:28:41.685163
1000 Evaluations: Average connections for default Uniform board = 4.048, std=0.8908961780140264
2023-04-25 20:29:12.124470


2023-04-15 20:04:46.888987
Average connections for default Uniform board = 4.14, std=0.8834025130143109
2023-04-15 20:05:35.859778

1000 evals = 7 min
2023-04-15 23:32:21.077616
Average connections for default Uniform board = 4.048, std=0.8908961780140264
2023-04-15 23:39:18.525824

10000 evals = 1hr 9 min 37sec
2023-04-15 23:47:56.598269
Average connections for default Uniform board = 4.0287, std=0.8960336545018832
2023-04-16 00:57:33.757044


## Rollout a few episodes of each of our board generators

In [11]:
# Keep this cell


step_fn = jax.jit(env.step) # Speed up env.step
BOARD_GENERATORS = [
        BoardName.BFS_BASE,
        BoardName.BFS_MIN_BENDS,
        BoardName.BFS_FIFO,
        BoardName.BFS_SHORTEST,
        BoardName.BFS_LONGEST,
        BoardName.RANDOM_WALK,
        BoardName.RANDOM_SEED,
        BoardName.LSYSTEMS,
        BoardName.WFC,
        BoardName.NUMBERLINK]

GRID_SIZE = 10
NUM_AGENTS = 5

NUM_EPISODES = 10 # 10000

print(datetime.now())
states = []
key = jax.random.PRNGKey(cfg.seed)

for BOARD_GENERATOR in [BoardName.RANDOM_SEED]:   # BOARD_GENERATORS:
    print(BOARD_GENERATOR) 
    for episode in range(NUM_EPISODES):
   
        key, reset_key = jax.random.split(key)
        #### Copied from ic_rl_training ###################################
        # Create empty grid.
        grid = jnp.zeros((GRID_SIZE, GRID_SIZE), dtype=jnp.int32)

        board_class = BoardGenerator.get_board_generator(
            board_enum=BOARD_GENERATOR)    

        if (BOARD_GENERATOR == BoardName.RANDOM_SEED):
            key, subkey = jax.random.split(key)
            board = RandomSeedBoard(GRID_SIZE, GRID_SIZE, NUM_AGENTS)
            jaxd_board_init = jax.jit(board.return_solved_board)
            solved_board = jaxd_board_init(subkey)
            #key, subkey = jax.random.split(key)
            #solved_board = jaxd_board_init(subkey)
            #solved_board = jax.jit(board.return_solved_board)(subkey)
            #print(solved_board)
            #starts, targets = jax.jit(board.generate_starts_ends)(subkey)
            #print(starts, targets)
            #pins = jax.jit(board.return_training_board)(subkey)
            #print(pins)
        else:
            board = board_class(GRID_SIZE, GRID_SIZE, NUM_AGENTS)
            print(board.return_solved_board())
            pins = board.return_training_board()
        
        starts_flat, targets_flat = get_heads_and_targets(pins)
        starts = jnp.divmod(starts_flat, GRID_SIZE)
        targets = jnp.divmod(targets_flat, GRID_SIZE)

        agent_position_values = jax.vmap(get_position)(jnp.arange(NUM_AGENTS))
        agent_target_values = jax.vmap(get_target)(jnp.arange(NUM_AGENTS))

        # Place the agent values at starts and targets.
        grid = grid.at[starts].set(agent_position_values)
        grid = grid.at[targets].set(agent_target_values)

        # Create the agent pytree that corresponds to the grid.
        agents = jax.vmap(Agent)(
            id=jnp.arange(NUM_AGENTS),
            start=jnp.stack(starts, axis=1),
            target=jnp.stack(targets, axis=1),
            position=jnp.stack(starts, axis=1),
        )
        step_count = jnp.array(0, jnp.int32)
        state = State(key=key, grid=grid, step_count=step_count, agents=agents)
        ### END OF CODE FROM IC_RL_TRAINING  ##################################
       
        
        ### AMENDMENT 2 FROM CLEMENT TO INITIALIZE TIMESTEP #################
        action_mask = jax.vmap(env._get_action_mask, (0, None))(
            state.agents, state.grid
            )
        observation = Observation(
            grid=env._obs_from_grid(state.grid),
            action_mask=action_mask,
            step_count=state.step_count,
            )
        extras = env._get_extras(state)
        timestep = jumanji.types.restart(
            observation=observation, extras=extras, shape=(env.num_agents,)
            )
        ####################################################################
        
        while not timestep.last():
            key, action_key = jax.random.split(key)
            observation = jax.tree_util.tree_map(lambda x: x[None], timestep.observation)
            # Two implementations for calling the policy, about equivalent speed
            #action, _ = policy(observation, action_key)
            action, _ = jax.jit(policy)(observation, action_key)
            # Three implementations for updating the state/timestep.  The third is much faster.
            #state, timestep = jax.jit(env.step)(state, action.squeeze(axis=0)) # original jit = 0.32, 52sec/10
            #state, timestep = env.step(state, action.squeeze(axis=0)) # no jit = 0.13, 26sec/10
            state, timestep = step_fn(state, action.squeeze(axis=0)) # jit function = 0.003 5 sec/10, 49sec/100d
            states.append(state)
        # Freeze the terminal frame to pause the GIF.
        for _ in range(10):
            states.append(state)
            
        # Evaluate the number of wires connected
        num_connected = timestep.extras["num_connections"]
        connections.append(num_connected)
    connections = np.array(connections)
    print(f"{NUM_EPISODES} Evaluations: Average connections for {BOARD_GENERATOR} = {connections.mean()}, std={connections.std()}")
        
    print(datetime.now())

2023-04-25 22:34:07.051112
BoardName.RANDOM_SEED


TypeError: divmod requires ndarray or scalar arguments, got <class 'list'> at position 0.

In [52]:
# Extend the randomseed in this cell
from ic_routing_board_generation.board_generator.board_generator_random_seed_rb import RandomSeedBoard

step_fn = jax.jit(env.step) # Speed up env.step
BOARD_GENERATORS = [
        BoardName.BFS_BASE,
        BoardName.BFS_MIN_BENDS,
        BoardName.BFS_FIFO,
        BoardName.BFS_SHORTEST,
        BoardName.BFS_LONGEST,
        BoardName.RANDOM_WALK,
        BoardName.RANDOM_SEED,
        BoardName.LSYSTEMS,
        BoardName.WFC,
        BoardName.NUMBERLINK]

GRID_SIZE = 10
NUM_AGENTS = 5

NUM_EPISODES = 1000 # 10000

print(datetime.now())
states = []
connections = []
key = jax.random.PRNGKey(cfg.seed)

for BOARD_GENERATOR in [BoardName.RANDOM_SEED]:   # BOARD_GENERATORS:
    print(BOARD_GENERATOR)         
    if (BOARD_GENERATOR == BoardName.RANDOM_SEED):
        board = RandomSeedBoard(GRID_SIZE, GRID_SIZE, NUM_AGENTS)
        #jaxd_board_init = jax.jit(board.return_solved_board)
        #extended_board_init = jax.jit(board.return_extended_board)

    for episode in range(NUM_EPISODES):
   
        key, reset_key = jax.random.split(key)
        #### Copied from ic_rl_training ###################################
        # Create empty grid.
        grid = jnp.zeros((GRID_SIZE, GRID_SIZE), dtype=jnp.int32)

        board_class = BoardGenerator.get_board_generator(
            board_enum=BOARD_GENERATOR)    

        if (BOARD_GENERATOR == BoardName.RANDOM_SEED):
            key, subkey = jax.random.split(key)
            solved_board = jaxd_board_init(subkey)
            #print(solved_board)
            #extended_board_straight = extended_board_init(subkey, 0.0)
            #print(extended_board_straight)
            #extended_board_5050 = extended_board_init(subkey, 0.5)
            #print(extended_board_5050)
            #extended_board_random = extended_board_init(subkey, 1.0)
            #print(extended_board_random)
            extended_board = extended_board_init(subkey, 0.0)
            #starts, targets = jax.jit(board.generate_starts_ends)(subkey)
            #print(starts, targets)
            #pins = jax.jit(board.return_training_board)(subkey)
            #print(pins)
            pins = np.array(extended_board)
            rows, cols = extended_board.shape
            for row in range(rows):
                for col in range (cols):
                    if (pins[row, col] % 3) == 1:  # PATH cell
                        pins[row, col] = 0
            #print(pins)
        else:
            board = board_class(GRID_SIZE, GRID_SIZE, NUM_AGENTS)
            print(board.return_solved_board())
            pins = board.return_training_board()
        
        starts_flat, targets_flat = get_heads_and_targets(pins)
        starts = jnp.divmod(starts_flat, GRID_SIZE)
        targets = jnp.divmod(targets_flat, GRID_SIZE)

        agent_position_values = jax.vmap(get_position)(jnp.arange(NUM_AGENTS))
        agent_target_values = jax.vmap(get_target)(jnp.arange(NUM_AGENTS))

        # Place the agent values at starts and targets.
        grid = grid.at[starts].set(agent_position_values)
        grid = grid.at[targets].set(agent_target_values)

        # Create the agent pytree that corresponds to the grid.
        agents = jax.vmap(Agent)(
            id=jnp.arange(NUM_AGENTS),
            start=jnp.stack(starts, axis=1),
            target=jnp.stack(targets, axis=1),
            position=jnp.stack(starts, axis=1),
        )
        step_count = jnp.array(0, jnp.int32)
        state = State(key=key, grid=grid, step_count=step_count, agents=agents)
        ### END OF CODE FROM IC_RL_TRAINING  ##################################
       
        
        ### AMENDMENT 2 FROM CLEMENT TO INITIALIZE TIMESTEP #################
        action_mask = jax.vmap(env._get_action_mask, (0, None))(
            state.agents, state.grid
            )
        observation = Observation(
            grid=env._obs_from_grid(state.grid),
            action_mask=action_mask,
            step_count=state.step_count,
            )
        extras = env._get_extras(state)
        timestep = jumanji.types.restart(
            observation=observation, extras=extras, shape=(env.num_agents,)
            )
        ####################################################################
        
        while not timestep.last():
            key, action_key = jax.random.split(key)
            observation = jax.tree_util.tree_map(lambda x: x[None], timestep.observation)
            # Two implementations for calling the policy, about equivalent speed
            #action, _ = policy(observation, action_key)
            action, _ = jax.jit(policy)(observation, action_key)
            # Three implementations for updating the state/timestep.  The third is much faster.
            #state, timestep = jax.jit(env.step)(state, action.squeeze(axis=0)) # original jit = 0.32, 52sec/10
            #state, timestep = env.step(state, action.squeeze(axis=0)) # no jit = 0.13, 26sec/10
            state, timestep = step_fn(state, action.squeeze(axis=0)) # jit function = 0.003 5 sec/10, 49sec/100d
            states.append(state)
        # Freeze the terminal frame to pause the GIF.
        for _ in range(10):
            states.append(state)
            
        # Evaluate the number of wires connected
        num_connected = timestep.extras["num_connections"]
        connections.append(num_connected)
    connections = np.array(connections)
    print(f"{NUM_EPISODES} Evaluations: Average connections for {BOARD_GENERATOR} = {connections.mean()}, std={connections.std()}")
        
    print(datetime.now())

2023-04-20 22:20:19.895471
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 4.454, std=0.7401918670182752
2023-04-20 22:21:44.037252


SAVE SOME RESULTS, HOW THE "UNIFORM AGENT" PERFORMS ON 
RANDOMSEED-EXTEND-RANDOM
2023-04-20 22:12:47.813589
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 4.407, std=0.7164851708165355
2023-04-20 22:14:08.681154

RANDOMSEED-EXTEND-STRAIGHT
2023-04-20 22:20:19.895471
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 4.454, std=0.7401918670182752
2023-04-20 22:21:44.037252


10000 evals = 1hr 8min 11sec/5 = 14min/generator
2023-04-16 00:57:33.807103
BoardName.BFS_BASE
Average connections for bfs_base = 4.5573, std=0.6149119530469382
2023-04-16 01:12:05.259665
BoardName.BFS_MIN_BENDS
Average connections for bfs_min_bend = 4.5019, std=0.6561984379743676
2023-04-16 01:25:57.235825
BoardName.BFS_FIFO
Average connections for bfs_fifo = 4.6212, std=0.5790600659689804
2023-04-16 01:39:11.518756
BoardName.BFS_SHORTEST
Average connections for bfs_short = 4.4307, std=0.6763116958917685
2023-04-16 01:52:50.729754
BoardName.BFS_LONGEST
Average connections for bfs_long = 4.717, std=0.5172146556314892
2023-04-16 02:05:44.179665
BoardName.RANDOM_WALK

10000 Evals = 2hr 3min 27sec
WFC = 1hr 5min 19sec, other four =58/4 = 14.5min/generator
2023-04-16 07:54:41.199665
BoardName.BFS_LONGEST
10000 Evaluation: Average connections for default Uniform board = 4.7163, std=0.5147954059624076
Average connections for bfs_long = 4.7163, std=0.5147954059624076
2023-04-16 08:07:22.442150
BoardName.RANDOM_WALK
10000 Evaluation: Average connections for default Uniform board = 4.676, std=0.5460988921431722
Average connections for random_walk = 4.676, std=0.5460988921431722
2023-04-16 08:19:17.594845
BoardName.LSYSTEMS_STANDARD
10000 Evaluation: Average connections for default Uniform board = 4.9944, std=0.07725697379525037
Average connections for lsystems_standard = 4.9944, std=0.07725697379525037
2023-04-16 08:28:43.780898
BoardName.WFC
10000 Evaluation: Average connections for default Uniform board = 4.833, std=0.4085474268674324
Average connections for wfc = 4.833, std=0.4085474268674324
2023-04-16 09:34:02.631164
BoardName.NUMBERLINK
10000 Evaluation: Average connections for default Uniform board = 4.0041, std=0.9051426351686236
Average connections for numberlink = 4.0041, std=0.9051426351686236
2023-04-16 09:58:08.309304

# Roll out a few episodes, extending the wires of each board generator

In [10]:
#WE PROBABLY DON'T NEED TO EXTEND THE WIRES OF EACH ANY MORE

step_fn = jax.jit(env.step) #  Speed up env.step
BOARD_GENERATORS = [
        BoardName.BFS_BASE,
        BoardName.BFS_MIN_BENDS,
        BoardName.BFS_FIFO,
        BoardName.BFS_SHORTEST,
        BoardName.BFS_LONGEST,
        BoardName.RANDOM_WALK,
        BoardName.LSYSTEMS,
        BoardName.WFC,
        BoardName.NUMBERLINK]

GRID_SIZE = 10
NUM_AGENTS = 5
NUM_EPISODES = 1 # 10000

print(datetime.now())
states = []
key = jax.random.PRNGKey(cfg.seed)

for BOARD_GENERATOR in BOARD_GENERATORS:
    print(BOARD_GENERATOR)
    connections = []
    detours_raw = []
    detours_extended = []
    for episode in range(NUM_EPISODES):
   
        key, reset_key = jax.random.split(key)


        ### Copied from ic_rl_training  ##########################
        # Create empty grid.
        grid = jnp.zeros((GRID_SIZE, GRID_SIZE), dtype=jnp.int32)

        board_class = BoardGenerator.get_board_generator(
            board_enum=BOARD_GENERATOR)    
        board = board_class(GRID_SIZE, GRID_SIZE, NUM_AGENTS)
        pins = board.return_training_board()

        ### WIRE EXTENSION #####
        #print("PINS")
        #print(pins)
        board_solved = board.return_solved_board()
        detours_raw.append(count_detours(board_solved, False))
        #print("BOARD SOLVED")
        #print(board_solved)
        board_extended = extend_wires(board_solved)
        detours_extended.append(count_detours(board_extended, False))
        #print("BOARD EXTENDED")
        #print(board_extended)
        pins = training_board_from_solved_board_jax(board_extended)
        #print("PINS EXTENDED")
        #print(pins)
        ### END OF WIRE EXTENSION ####

        starts_flat, targets_flat = get_heads_and_targets(pins)
        starts = jnp.divmod(starts_flat, GRID_SIZE)
        targets = jnp.divmod(targets_flat, GRID_SIZE)

        agent_position_values = jax.vmap(get_position)(jnp.arange(NUM_AGENTS))
        agent_target_values = jax.vmap(get_target)(jnp.arange(NUM_AGENTS))

        # Place the agent values at starts and targets.
        grid = grid.at[starts].set(agent_position_values)
        grid = grid.at[targets].set(agent_target_values)

        # Create the agent pytree that corresponds to the grid.
        agents = jax.vmap(Agent)(
            id=jnp.arange(NUM_AGENTS),
            start=jnp.stack(starts, axis=1),
            target=jnp.stack(targets, axis=1),
            position=jnp.stack(starts, axis=1),
        )

        step_count = jnp.array(0, jnp.int32)

        state = State(key=key, grid=grid, step_count=step_count, agents=agents)
        # END OF CODE FROM IC_RL_TRAINING
        ########################################################################################

        
        ###############AMENDMENT 2 FROM CLEMENT TO INITIALIZE TIMESTEP############################
        action_mask = jax.vmap(env._get_action_mask, (0, None))(
            state.agents, state.grid
            )
        observation = Observation(
            grid=env._obs_from_grid(state.grid),
            action_mask=action_mask,
            step_count=state.step_count,
            )
        extras = env._get_extras(state)
        timestep = jumanji.types.restart(
            observation=observation, extras=extras, shape=(env.num_agents,)
            )
        ####################################################################
        
        
        while not timestep.last():
            key, action_key = jax.random.split(key)
            observation = jax.tree_util.tree_map(lambda x: x[None], timestep.observation)
            # Two implementations for calling the policy, about equivalent speed
            #action, _ = policy(observation, action_key)
            action, _ = jax.jit(policy)(observation, action_key)
            # Three implementations for updating the state/timestep.  The third is much faster.
            #state, timestep = jax.jit(env.step)(state, action.squeeze(axis=0)) # original jit = 0.32, 52sec/10
            #state, timestep = env.step(state, action.squeeze(axis=0)) # no jit = 0.13, 26sec/10
            state, timestep = step_fn(state, action.squeeze(axis=0)) # jit function = 0.003 5 sec/10, 49sec/100d
            states.append(state)
        # Freeze the terminal frame to pause the GIF.
        for _ in range(10):
            states.append(state)

        # Evaluate the number of wires connected
        num_connected = timestep.extras["num_connections"]
        connections.append(num_connected)

    connections = np.array(connections)
    print(f"{NUM_EPISODES} Evaluations: Average connections for {BOARD_GENERATOR} = {connections.mean()}, std={connections.std()}")
    detours_raw = np.array(detours_raw)    
    print(f"Average Detours (RAW) = {detours_raw.mean()}, (std = {detours_raw.std()}) ")
    detours_extended = np.array(detours_extended)
    print(f"Average Detours (extended) = {detours_extended.mean()}, (std = {detours_extended.std()}) ")
    print(datetime.now())

2023-04-19 14:41:43.131017
BoardName.BFS_BASE
1 Evaluations: Average connections for bfs_base = 5.0, std=0.0
Average Detours (RAW) = 0.0, (std = 0.0) 
Average Detours (extended) = 1.0, (std = 0.0) 
2023-04-19 14:41:45.176461
BoardName.BFS_MIN_BENDS
1 Evaluations: Average connections for bfs_min_bend = 5.0, std=0.0
Average Detours (RAW) = 1.0, (std = 0.0) 
Average Detours (extended) = 2.0, (std = 0.0) 
2023-04-19 14:41:45.321709
BoardName.BFS_FIFO
1 Evaluations: Average connections for bfs_fifo = 5.0, std=0.0
Average Detours (RAW) = 0.0, (std = 0.0) 
Average Detours (extended) = 8.0, (std = 0.0) 
2023-04-19 14:41:45.461969
BoardName.BFS_SHORTEST
1 Evaluations: Average connections for bfs_short = 5.0, std=0.0
Average Detours (RAW) = 2.0, (std = 0.0) 
Average Detours (extended) = 2.0, (std = 0.0) 
2023-04-19 14:41:45.580676
BoardName.BFS_LONGEST
1 Evaluations: Average connections for bfs_long = 5.0, std=0.0
Average Detours (RAW) = 0.0, (std = 0.0) 
Average Detours (extended) = 0.0, (std =

SAVING SOME TEST RESULTS

10000 evals, L-systems
2023-04-17 13:20:00.211418
BoardName.LSYSTEMS_STANDARD
10000 Evaluations: Average connections for lsystems_standard = 4.3941, std=0.7488559207217367
2023-04-17 13:36:50.571677

1000 evals, all our boards
2023-04-17 13:53:54.225413
BoardName.BFS_BASE
1000 Evaluations: Average connections for bfs_base = 4.202, std=0.8019950124533194
2023-04-17 13:55:53.485387
BoardName.BFS_MIN_BENDS
1000 Evaluations: Average connections for bfs_min_bend = 4.08, std=0.8874683092933515
2023-04-17 13:58:03.262815
BoardName.BFS_FIFO
1000 Evaluations: Average connections for bfs_fifo = 4.19, std=0.8293973715897585
2023-04-17 14:00:14.402026
BoardName.BFS_SHORTEST
1000 Evaluations: Average connections for bfs_short = 4.109, std=0.8712743540355128
2023-04-17 14:02:29.357893
BoardName.BFS_LONGEST
1000 Evaluations: Average connections for bfs_long = 4.251, std=0.8087020464917842
2023-04-17 14:04:44.767001
BoardName.RANDOM_WALK
1000 Evaluations: Average connections for random_walk = 4.478, std=0.6822873295027544
2023-04-17 14:06:33.586283
BoardName.LSYSTEMS_STANDARD
1000 Evaluations: Average connections for lsystems_standard = 4.385, std=0.7475125417008066
2023-04-17 14:08:31.457369
BoardName.WFC
1000 Evaluations: Average connections for wfc = 4.53, std=0.6731270311018568
2023-04-17 14:15:43.012756
BoardName.NUMBERLINK
1000 Evaluations: Average connections for numberlink = 4.038, std=0.9124450668396425
2023-04-17 14:18:42.103116


10000 Evals
2023-04-18 00:19:31.909775
BoardName.BFS_BASE
10000 Evaluations: Average connections for bfs_base = 4.1471, std=0.8551383455324642
Average Detours (RAW) = 1.3449, (std = 1.8076349161265943) 
Average Detours (RAW) = 5.5036, (std = 3.7273834039443807) 
2023-04-18 00:38:01.948714
BoardName.BFS_MIN_BENDS
10000 Evaluations: Average connections for bfs_min_bend = 4.1253, std=0.8566212173417138
Average Detours (RAW) = 1.9149, (std = 2.1955085948362854) 
Average Detours (RAW) = 5.9369, (std = 3.7803066529052907) 
2023-04-18 00:58:03.050593
BoardName.BFS_FIFO
10000 Evaluations: Average connections for bfs_fifo = 4.1664, std=0.854699385749165
Average Detours (RAW) = 1.4981, (std = 1.9647891464480356) 
Average Detours (RAW) = 5.5746, (std = 3.694703620048569) 
2023-04-18 01:17:46.139231
BoardName.BFS_SHORTEST
10000 Evaluations: Average connections for bfs_short = 4.101, std=0.8699419520864596
Average Detours (RAW) = 1.6461, (std = 1.9280702243435013) 
Average Detours (RAW) = 5.6776, (std = 3.646677698947358) 
2023-04-18 01:37:43.132813
BoardName.BFS_LONGEST






ALSO WITH COMPARISION OF NUM_DETOURS

2023-04-17 14:41:57.356808
BoardName.BFS_BASE
10 Evaluations: Average connections for bfs_base = 4.0, std=0.8944271909999159
Average Detours (RAW) = 0.9, (std = 1.5132745950421556) 
Average Detours (RAW) = 5.0, (std = 2.7202941017470885) 
2023-04-17 14:41:58.981720
BoardName.BFS_MIN_BENDS
10 Evaluations: Average connections for bfs_min_bend = 4.2, std=1.1661903789690602
Average Detours (RAW) = 2.2, (std = 1.661324772583615) 
Average Detours (RAW) = 5.4, (std = 1.6852299546352716) 
2023-04-17 14:42:00.139787
BoardName.BFS_FIFO
10 Evaluations: Average connections for bfs_fifo = 4.6, std=0.6633249580710799
Average Detours (RAW) = 1.5, (std = 2.5) 
Average Detours (RAW) = 5.0, (std = 3.9749213828703582) 
2023-04-17 14:42:01.258899
BoardName.BFS_SHORTEST
10 Evaluations: Average connections for bfs_short = 3.9, std=1.2206555615733705
Average Detours (RAW) = 2.9, (std = 3.5902646142032486) 
Average Detours (RAW) = 6.8, (std = 4.791659420284375) 
2023-04-17 14:42:02.465300
BoardName.BFS_LONGEST
10 Evaluations: Average connections for bfs_long = 4.4, std=0.8
Average Detours (RAW) = 1.1, (std = 1.5132745950421556) 
Average Detours (RAW) = 5.4, (std = 2.939387691339814) 
2023-04-17 14:42:03.647215
BoardName.RANDOM_WALK
10 Evaluations: Average connections for random_walk = 4.3, std=0.7810249675906654
Average Detours (RAW) = 3.1, (std = 1.7) 
Average Detours (RAW) = 4.3, (std = 2.0518284528683193) 
2023-04-17 14:42:04.609534
BoardName.LSYSTEMS_STANDARD
10 Evaluations: Average connections for lsystems_standard = 4.1, std=1.1357816691600546
Average Detours (RAW) = 0.0, (std = 0.0) 
Average Detours (RAW) = 2.6, (std = 1.5620499351813308) 
2023-04-17 14:42:05.545080
BoardName.WFC
10 Evaluations: Average connections for wfc = 4.3, std=0.6403124237432849
Average Detours (RAW) = 1.2, (std = 1.32664991614216) 
Average Detours (RAW) = 2.5, (std = 1.857417562100671) 
2023-04-17 14:42:09.540836
BoardName.NUMBERLINK
10 Evaluations: Average connections for numberlink = 4.2, std=0.9797958971132713
Average Detours (RAW) = 16.7, (std = 7.5504966724050675) 
Average Detours (RAW) = 16.7, (std = 7.5504966724050675) 
2023-04-17 14:42:11.523953

SAVING SOME METRICS OF RANDOMNESS VS. AVERAGE DETOURS

2023-04-20 07:25:39.407550
BoardName.RANDOM_SEED
10 Evaluations: Average connections for random_seed = 4.4, std=0.4898979485566356
Average Detours (RAW) = 0.0, (std = 0.0) 
Average Detours (extended_jax000) = 4.7, (std = 2.968164415931166) 
Average Detours (extended_jax050) = 4.3, (std = 1.9000000000000001) 
Average Detours (extended_jax100) = 2.9, (std = 2.118962010041709) 
2023-04-20 07:26:09.907502


SAVING SOME TIMING RESULTS
TIME TO CREATE 100 RANDOMSEED BOARDS AND EXTEND THEM THREE WAYS (NO SOLVING) = 27 SEC
2023-04-20 07:18:23.928470
BoardName.RANDOM_SEED
Average Detours (RAW) = 0.0, (std = 0.0) 
Average Detours (extended_jax000) = 6.8, (std = 4.791659420284375) 
Average Detours (extended_jax050) = 4.3, (std = 4.382921400162225) 
Average Detours (extended_jax100) = 2.8, (std = 1.2489995996796797) 
2023-04-20 07:18:50.919427

TIME TO CREATE 100 RANDOMSEED BOARDS AND EXTEND THEM THREE WAYS AND SOLVE THEM =  28 SEC
2023-04-20 07:20:13.289647
BoardName.RANDOM_SEED
10 Evaluations: Average connections for random_seed = 4.4, std=0.4898979485566356
Average Detours (RAW) = 0.0, (std = 0.0) 
Average Detours (extended_jax000) = 4.7, (std = 2.968164415931166) 
Average Detours (extended_jax050) = 4.3, (std = 1.9000000000000001) 
Average Detours (extended_jax100) = 2.9, (std = 2.118962010041709) 
2023-04-20 07:20:41.051680

TIME TO CREATE 100 RANDOMSEED BOARDS AND EXTEND THEM ONE WAY AND SOLVE THEM = 32?? SEC
2023-04-20 07:23:40.554055
BoardName.RANDOM_SEED
10 Evaluations: Average connections for random_seed = 4.4, std=0.4898979485566356
Average Detours (RAW) = 0.0, (std = 0.0) 
Average Detours (extended_jax000) = nan, (std = nan) 
Average Detours (extended_jax050) = nan, (std = nan) 
Average Detours (extended_jax100) = 2.9, (std = 2.118962010041709) 
2023-04-20 07:24:08.715292






SAVING SOME TEST RESULTS
2023-04-18 23:30:20.047379
BoardName.BFS_BASE
10000 Evaluations: Average connections for bfs_base = 4.161, std=0.8491637062427952
Average Detours (RAW) = 1.3723, (std = 1.8170010209133072) 
Average Detours (extended) = 5.4804, (std = 3.656475877125405) 
Average Detours (extended_jax) = 5.4839, (std = 3.668942734630782) 
2023-04-18 23:49:23.378747
BoardName.BFS_MIN_BENDS
10000 Evaluations: Average connections for bfs_min_bend = 4.1164, std=0.8631633912533593
Average Detours (RAW) = 1.9133, (std = 2.181417683526014) 
Average Detours (extended) = 5.8725, (std = 3.7469245722325395) 
Average Detours (extended_jax) = 5.8499, (std = 3.727998120975921) 
2023-04-19 00:09:38.284441
BoardName.BFS_FIFO

2023-04-19 07:17:48.746798
BoardName.BFS_FIFO
Traced<ShapedArray(int32[10,10])>with<DynamicJaxprTrace(level=2/0)>
10000 Evaluations: Average connections for bfs_fifo = 4.1725, std=0.8403235983833847
Average Detours (RAW) = 1.4899, (std = 1.9635422047921456) 
Average Detours (extended) = 5.6108, (std = 3.7632862447600233) 
Average Detours (extended_jax) = 5.6157, (std = 3.7834658066381412) 
2023-04-19 07:40:39.928428
BoardName.BFS_SHORTEST
10000 Evaluations: Average connections for bfs_short = 4.1121, std=0.8617038876551504
Average Detours (RAW) = 1.6072, (std = 1.8837484333105627) 
Average Detours (extended) = 5.6431, (std = 3.6416922426256724) 
Average Detours (extended_jax) = 5.6279, (std = 3.6718716739559403) 
2023-04-19 08:00:57.713862
BoardName.BFS_LONGEST
10000 Evaluations: Average connections for bfs_long = 4.2254, std=0.8283687826083259
Average Detours (RAW) = 0.9742, (std = 1.6056569870305426) 
Average Detours (extended) = 5.1962, (std = 3.6904343321620017) 
Average Detours (extended_jax) = 5.1925, (std = 3.694677759967708) 
2023-04-19 08:21:09.342983
BoardName.RANDOM_WALK

2023-04-19 09:34:00.124245
BoardName.RANDOM_WALK
Traced<ShapedArray(int32[10,10])>with<DynamicJaxprTrace(level=2/0)>
10000 Evaluations: Average connections for random_walk = 4.4929, std=0.6710809116641598
Average Detours (RAW) = 3.2179, (std = 2.296392734268248) 
Average Detours (extended) = 4.9207, (std = 3.020266794506737) 
Average Detours (extended_jax) = 4.9243, (std = 3.026742392408049) 
2023-04-19 09:53:42.164757
BoardName.LSYSTEMS_STANDARD
10000 Evaluations: Average connections for lsystems_standard = 4.378, std=0.7486761649738825
Average Detours (RAW) = 0.0133, (std = 0.12539182588988804) 
Average Detours (extended) = 3.6276, (std = 2.6439209973068407) 
Average Detours (extended_jax) = 3.6961, (std = 2.642450527446068) 
2023-04-19 10:11:11.249583
BoardName.WFC
10000 Evaluations: Average connections for wfc = 4.5191, std=0.6747111900657939
Average Detours (RAW) = 0.8632, (std = 1.434672701350381) 
Average Detours (extended) = 3.174, (std = 2.7402415951882784) 
Average Detours (extended_jax) = 3.1766, (std = 2.724447180622153) 
2023-04-19 11:22:28.078346
BoardName.NUMBERLINK



IT'S FASTER WITHOUT THE JAX.JIT() CALLS IN THE WIRE EXTENSION

JAX.JIT() CALL APPLIED TO BOTH THE EXTENSION AND PIN EXTRACTION
2023-04-18 23:17:35.299726
BoardName.RANDOM_WALK
1000 Evaluations: Average connections for random_walk = 4.514, std=0.6795616234014396
2023-04-18 23:22:00.377340

JAX.JIT() CALL APPLIED TO JUST THE PIN EXTRACTION
2023-04-18 23:22:53.809285
BoardName.RANDOM_WALK
1000 Evaluations: Average connections for random_walk = 4.486, std=0.6721636705446077
2023-04-18 23:24:32.408968

JAX.JIT() CALL APPLIED TO NEITHER
2023-04-18 23:25:10.222635
BoardName.RANDOM_WALK
1000 Evaluations: Average connections for random_walk = 4.484, std=0.655548625198772
2023-04-18 23:26:47.720919



## Save GIF

In [ ]:
#env.animate(states, interval=150).save("./connector.gif")

## Save PNG

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

#env.render(states[-1])
state.grid = solved_board
env.render(state)
#plt.savefig("/home/randy/Downloads/randomseed_extended_jax00.png", dpi=300)

def correlation(x,y):
    dx = x-x.mean()
    dy = y-y.mean()
    correlation = (dx * dy).sum() / sqrt((dx**2).sum * (dy**2).sum())
    return correlation
    

In [16]:
# TEST ALL THE AGENTS ON ALL THE BOARDS
import statistics
  
key = jax.random.PRNGKey(cfg.seed)
step_fn = jax.jit(env.step) # Speed up env.step
BOARD_GENERATORS = [
        BoardName.BFS_BASE,
        #BoardName.BFS_MIN_BENDS,
        #BoardName.BFS_FIFO,
        BoardName.BFS_SHORTEST,
        BoardName.BFS_LONGEST,
        BoardName.RANDOM_WALK,
        BoardName.RANDOM_SEED,
        #BoardName.LSYSTEMS,
        #BoardName.WFC,
        BoardName.NUMBERLINK]

agents = ["uniform", "bfs_short", "random_walk", "numberlink", "lsystems"]  #file = "examples/"+agent+"/training_state"
#agents = ["iterations1","iterations2","iterations5","iterations6","iterations8"] #file = "examples/fullsteps_random1/"+agentname+"/training_state"

GRID_SIZE = 10
NUM_AGENTS = 5
NUM_EPISODES = 1000 # 10000


for agentname in agents:
    file = "examples/"+agentname+"/training_state"
    #file = "examples/fullsteps_random1/"+agentname+"/training_state"
    print("\nAGENT = ",agentname)
    with open(file,"rb") as f:
        training_state = pickle.load(f)
    params = first_from_device(training_state.params_state.params)
    env = setup_env(cfg).unwrapped
    agent = setup_agent(cfg, env)
    policy = jax.jit(agent.make_policy(params.actor, stochastic = False))
    
    connections_all = []
    detours_all = []
    keys_all = []
    episodes_all = []
    densities_all = []
    print(datetime.now())
    #for BOARD_GENERATOR in BOARD_GENERATORS: # restore this
    BOARD_GENERATOR = BoardName.RANDOM_SEED # delete this
    for iterations in range(11): # delete this
        print("ITERATIONS = ", randomness) # delete this
        
        states = []
        connections = []
        print(BOARD_GENERATOR)         
        if (BOARD_GENERATOR == BoardName.RANDOM_SEED):
            board = RandomSeedBoard(GRID_SIZE, GRID_SIZE, NUM_AGENTS)
            solved_board_compiled = jax.jit(board.return_solved_board, 
                    static_argnames=['extension_iterations', 'randomness','two_sided','extension_steps'])
            #training_board_compiled = jax.jit(board.return_training_board)
            #starts_targets_compiled = jax.jit(board.generate_starts_ends)

            #extended_board_init = board.return_extended_board  #jax.jit(board.return_extended_board)

        for episode in range(NUM_EPISODES):

            key, reset_key = jax.random.split(key)
            #### Copied from ic_rl_training ###################################
            # Create empty grid.
            grid = jnp.zeros((GRID_SIZE, GRID_SIZE), dtype=jnp.int32)

            board_class = BoardGenerator.get_board_generator(
                board_enum=BOARD_GENERATOR)    

            if (BOARD_GENERATOR == BoardName.RANDOM_SEED):
                key, subkey = jax.random.split(key)

                #solved_board = solved_board_compiled(subkey, randomness = 1.0, two_sided = True, 
                                                     #extension_iterations=21, extension_steps=1e23) 
                solved_board = solved_board_compiled(subkey, randomness = 0.85, two_sided = True, 
                                     extension_iterations=iterations, extension_steps=1e23)
                #print("SOLVED BOARD")
                #print(solved_board)
                training_board = training_board_from_solved_board(np.array(solved_board))
                pins = training_board
                #print("TRAINING BOARD")
                #print(training_board)

            elif BOARD_GENERATOR == BoardName.LSYSTEMS:
                board = board_class(GRID_SIZE, GRID_SIZE, NUM_AGENTS)
                key, subkey = jax.random.split(key)
                pins = board.return_training_board(subkey)
                solved_board = board.return_solved_board(subkey)

            else:
                board = board_class(GRID_SIZE, GRID_SIZE, NUM_AGENTS)
                #print("SOLVED BOARD")
                #print(board.return_solved_board())
                solved_board = board.return_solved_board()
                pins = board.return_training_board() 


            starts_flat, targets_flat = get_heads_and_targets(pins)
            starts = jnp.divmod(np.array(starts_flat), GRID_SIZE)
            targets = jnp.divmod(np.array(targets_flat), GRID_SIZE)

            agent_position_values = jax.vmap(get_position)(jnp.arange(NUM_AGENTS))
            agent_target_values = jax.vmap(get_target)(jnp.arange(NUM_AGENTS))

            # Place the agent values at starts and targets.
            grid = grid.at[starts].set(agent_position_values)
            grid = grid.at[targets].set(agent_target_values)

            # Create the agent pytree that corresponds to the grid.
            agents = jax.vmap(Agent)(
                id=jnp.arange(NUM_AGENTS),
                start=jnp.stack(starts, axis=1),
                target=jnp.stack(targets, axis=1),
                position=jnp.stack(starts, axis=1),
            )
            step_count = jnp.array(0, jnp.int32)
            state = State(key=key, grid=grid, step_count=step_count, agents=agents)
            ### END OF CODE FROM IC_RL_TRAINING  ##################################

            ### AMENDMENT 2 FROM CLEMENT TO INITIALIZE TIMESTEP #################
            action_mask = jax.vmap(env._get_action_mask, (0, None))(
                state.agents, state.grid
                )
            observation = Observation(
                grid=env._obs_from_grid(state.grid),
                action_mask=action_mask,
                step_count=state.step_count,
                )
            extras = env._get_extras(state)
            timestep = jumanji.types.restart(
                observation=observation, extras=extras, shape=(env.num_agents,)
                )
            ####################################################################
            # Let the agent try to solve each board
            while not timestep.last():
                key, action_key = jax.random.split(key)
                observation = jax.tree_util.tree_map(lambda x: x[None], timestep.observation)
                # Two implementations for calling the policy, about equivalent speed
                #action, _ = policy(observation, action_key)
                action, _ = jax.jit(policy)(observation, action_key)
                # Three implementations for updating the state/timestep.  The third is much faster.
                #state, timestep = jax.jit(env.step)(state, action.squeeze(axis=0)) # original jit = 0.32, 52sec/10
                #state, timestep = env.step(state, action.squeeze(axis=0)) # no jit = 0.13, 26sec/10
                state, timestep = step_fn(state, action.squeeze(axis=0)) # jit function = 0.003 5 sec/10, 49sec/100d
                states.append(state)
            # Freeze the terminal frame to pause the GIF.
            for _ in range(10):
                states.append(state)

            # Evaluate the number of wires connected
            num_connected = timestep.extras["num_connections"]
            connections.append(num_connected)
            connections_all.append(num_connected)
            keys_all.append(key[0])
            detours_all.append(count_detours(np.array(solved_board)))
            episodes_all.append(episode)
            densities_all.append(np.count_nonzero(np.array(solved_board)>0))
            # End of section for each individual board
            
        connections = np.array(connections)
        print(f"{NUM_EPISODES} Evaluations: Average connections for {BOARD_GENERATOR} = {connections.mean()}, std={connections.std()}")  
        print(datetime.now())
        # End of section for each board type
        
    connections_all = np.array(connections_all)
    print(f"{NUM_EPISODES} Evaluations: Total average connections for {agentname} = {connections_all.mean()}, std={connections_all.std()}")  
    print("CORRELATIONS")
    detours_all = np.array(detours_all)
    keys_all = np.array(keys_all)
    episodes_all = np.array(episodes_all)
    print("mean keys = ", keys_all.mean())
    print("mean episodes = ", episodes_all.mean())
    print("Correlation of connections with detours is ", statistics.correlation(connections_all, detours_all))
    print("Correlation of connections with keys is ", statistics.correlation(connections_all, keys_all))
    print("Correlation of connections with episodes is ", statistics.correlation(connections_all, episodes_all))
    print("Correlation of connections with densities is ", statistics.correlation(connections_all, densities_all))



AGENT =  uniform
2023-04-27 18:11:56.575567
RANDOMNESS =  0.7
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 3.872, std=0.9484808906878409
2023-04-27 18:14:14.296430
RANDOMNESS =  0.9
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 3.918, std=0.9007086099288716
2023-04-27 18:16:30.520812
1000 Evaluations: Total average connections for uniform = 3.895, std=0.9251891698458213
CORRELATIONS
mean keys =  2165980172.822
mean episodes =  499.5
Correlation of connections with detours is  0.023085121331560397
Correlation of connections with keys is  0.015290579963265286
Correlation of connections with episodes is  -0.03307635883175643
Correlation of connections with densities is  0.025520556996522312

AGENT =  bfs_short
2023-04-27 18:16:30.801156
RANDOMNESS =  0.7
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 2.467, std=1.1911805068922174
2023-04-27 18:19:08.944563
RANDOMNESS =  0.9
BoardName.RANDOM_SE

EFFECT OF VARYING RANDOMNESS ON DIFFICULTY OF BOARDS. USE THIS DATA.


AGENT =  uniform
2023-04-27 12:55:41.998945
RANDOMNESS =  0.0
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 4.007, std=0.879176319062337
2023-04-27 12:57:41.341095
RANDOMNESS =  0.2
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 3.919, std=0.9297521175022943
2023-04-27 12:59:45.443131
RANDOMNESS =  0.4
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 3.915, std=0.9032026350714439
2023-04-27 13:02:07.524502
RANDOMNESS =  0.6000000000000001
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 3.907, std=0.9393354033570755
2023-04-27 13:04:31.324506
RANDOMNESS =  0.8
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 3.849, std=0.9360550197504417
2023-04-27 13:06:56.647882
1000 Evaluations: Total average connections for uniform = 3.9194, std=0.9191864011178582
CORRELATIONS
mean keys =  2134481010.8688
mean episodes =  499.5
Correlation of connections with detours is  0.011305661655349226
Correlation of connections with keys is  -0.002744965683393545
Correlation of connections with episodes is  0.002253283326174068
Correlation of connections with densities is  0.09457313408172009

AGENT =  bfs_short
2023-04-27 13:06:57.150821
RANDOMNESS =  0.0
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 2.358, std=1.1974289122950057
2023-04-27 13:09:49.147055
RANDOMNESS =  0.2
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 2.392, std=1.1403227613268097
2023-04-27 13:12:40.668025
RANDOMNESS =  0.4
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 2.391, std=1.219884830629515
2023-04-27 13:15:38.545654
RANDOMNESS =  0.6000000000000001
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 2.419, std=1.1659498273939577
2023-04-27 13:18:32.515248
RANDOMNESS =  0.8
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 2.393, std=1.1800639813162674
2023-04-27 13:21:29.750624
1000 Evaluations: Total average connections for bfs_short = 2.3906, std=1.181199238062741
CORRELATIONS
mean keys =  2135186815.0442
mean episodes =  499.5
Correlation of connections with detours is  0.05061654618963962
Correlation of connections with keys is  -0.03389629015679985
Correlation of connections with episodes is  0.028171812249063607
Correlation of connections with densities is  0.04788987227295542

AGENT =  random_walk
2023-04-27 13:21:30.319617
RANDOMNESS =  0.0
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 3.632, std=1.1075089164426624
2023-04-27 13:24:05.067040
RANDOMNESS =  0.2
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 3.644, std=1.062668339605542
2023-04-27 13:26:46.108912
RANDOMNESS =  0.4
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 3.611, std=1.107103879498216
2023-04-27 13:29:29.431862
RANDOMNESS =  0.6000000000000001
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 3.545, std=1.1162325026624158
2023-04-27 13:32:10.120949
RANDOMNESS =  0.8
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 3.403, std=1.2052348318896198
2023-04-27 13:34:51.904573
1000 Evaluations: Total average connections for random_walk = 3.567, std=1.1242379641339282
CORRELATIONS
mean keys =  2127939578.2798
mean episodes =  499.5
Correlation of connections with detours is  -0.014017925754193488
Correlation of connections with keys is  0.006090195846566444
Correlation of connections with episodes is  -0.005220629874090065
Correlation of connections with densities is  0.05382389682632095

AGENT =  numberlink
2023-04-27 13:34:52.434972
RANDOMNESS =  0.0
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 3.735, std=1.0718092180980718
2023-04-27 13:37:37.922877
RANDOMNESS =  0.2
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 3.632, std=1.0847008804274108
2023-04-27 13:40:23.249398
RANDOMNESS =  0.4
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 3.574, std=1.1360123238768143
2023-04-27 13:43:09.970496
RANDOMNESS =  0.6000000000000001
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 3.535, std=1.1299446889118068
2023-04-27 13:45:54.603771
RANDOMNESS =  0.8
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 3.521, std=1.097068366146796
2023-04-27 13:48:41.511482
1000 Evaluations: Total average connections for numberlink = 3.5994, std=1.106941570273698
CORRELATIONS
mean keys =  2180908412.8164
mean episodes =  499.5
Correlation of connections with detours is  0.0037109097546735594
Correlation of connections with keys is  -0.020010883342768297
Correlation of connections with episodes is  0.010728021246474036
Correlation of connections with densities is  0.09119376643688956

AGENT =  lsystems
2023-04-27 13:48:42.055885
RANDOMNESS =  0.0
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 2.598, std=1.1867586106702577
2023-04-27 13:51:59.251921
RANDOMNESS =  0.2
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 2.476, std=1.1980918161810472
2023-04-27 13:55:17.974808
RANDOMNESS =  0.4
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 2.514, std=1.155769873287931
2023-04-27 13:58:35.263556
RANDOMNESS =  0.6000000000000001
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 2.523, std=1.1762104403549563
2023-04-27 14:01:27.846120
RANDOMNESS =  0.8
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 2.538, std=1.117388025709959
2023-04-27 14:04:04.997189
1000 Evaluations: Total average connections for lsystems = 2.5298, std=1.1678664135936094
CORRELATIONS
mean keys =  2131893109.6196
mean episodes =  499.5
Correlation of connections with detours is  0.04714590308728007
Correlation of connections with keys is  -0.02441240178206892
Correlation of connections with episodes is  0.01433407043001087
Correlation of connections with densities is  0.03334951497660978



AGENT =  uniform
2023-04-27 18:11:56.575567
RANDOMNESS =  0.7
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 3.872, std=0.9484808906878409
2023-04-27 18:14:14.296430
RANDOMNESS =  0.9
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 3.918, std=0.9007086099288716
2023-04-27 18:16:30.520812
1000 Evaluations: Total average connections for uniform = 3.895, std=0.9251891698458213
CORRELATIONS
mean keys =  2165980172.822
mean episodes =  499.5
Correlation of connections with detours is  0.023085121331560397
Correlation of connections with keys is  0.015290579963265286
Correlation of connections with episodes is  -0.03307635883175643
Correlation of connections with densities is  0.025520556996522312

AGENT =  bfs_short
2023-04-27 18:16:30.801156
RANDOMNESS =  0.7
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 2.467, std=1.1911805068922174
2023-04-27 18:19:08.944563
RANDOMNESS =  0.9
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 2.374, std=1.1610874213425963
2023-04-27 18:21:47.759389
1000 Evaluations: Total average connections for bfs_short = 2.4205, std=1.177148992269033
CORRELATIONS
mean keys =  2145194797.709
mean episodes =  499.5
Correlation of connections with detours is  0.07300163947756115
Correlation of connections with keys is  0.013474409527401065
Correlation of connections with episodes is  0.01102443042438937
Correlation of connections with densities is  0.017377207262664993

AGENT =  random_walk
2023-04-27 18:21:48.018993
RANDOMNESS =  0.7
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 3.49, std=1.1462547709824373
2023-04-27 18:24:06.992383
RANDOMNESS =  0.9
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 3.521, std=1.1070496827152791
2023-04-27 18:26:26.752183
1000 Evaluations: Total average connections for random_walk = 3.5055, std=1.1269293456113387
CORRELATIONS
mean keys =  2182137539.12
mean episodes =  499.5
Correlation of connections with detours is  0.028184032641976272
Correlation of connections with keys is  0.028354049429339154
Correlation of connections with episodes is  0.002699680132137368
Correlation of connections with densities is  0.03129020668939027

AGENT =  numberlink
2023-04-27 18:26:27.016710
RANDOMNESS =  0.7
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 3.507, std=1.1313491945460517
2023-04-27 18:28:55.831158
RANDOMNESS =  0.9
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 3.452, std=1.1089165883870618
2023-04-27 18:31:29.059450
1000 Evaluations: Total average connections for numberlink = 3.4795, std=1.1205265503324764
CORRELATIONS
mean keys =  2114775628.8135
mean episodes =  499.5
Correlation of connections with detours is  0.06145891857943215
Correlation of connections with keys is  0.0026367643456800187
Correlation of connections with episodes is  0.005481995280327615
Correlation of connections with densities is  0.03422973066199895

AGENT =  lsystems
2023-04-27 18:31:30.398550
RANDOMNESS =  0.7
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 2.453, std=1.146207223847416
2023-04-27 18:34:23.810978
RANDOMNESS =  0.9
BoardName.RANDOM_SEED
1000 Evaluations: Average connections for random_seed = 2.437, std=1.1532696995932912
2023-04-27 18:37:13.005637
1000 Evaluations: Total average connections for lsystems = 2.445, std=1.1497717164724481
CORRELATIONS
mean keys =  2103361542.432
mean episodes =  499.5
Correlation of connections with detours is  0.07091817898543215
Correlation of connections with keys is  -0.006427223689533816
Correlation of connections with episodes is  -0.030226536566152415
Correlation of connections with densities is  0.023879897009763826

In [ ]:
EFFECT OF INCREASING ITERATIONS (RANDOMNESS = 0.85)






In [ ]:
EFFECT OF VARYING RANDOMNESS ON DIFFICULTY OF BOARDS
BUT ONLY 100 EVALUATIONS.  NOT ENOUGH.

AGENT =  uniform
2023-04-27 12:29:42.481720
RANDOMNESS =  0.0
BoardName.RANDOM_SEED
100 Evaluations: Average connections for random_seed = 4.07, std=0.851528038293514
2023-04-27 12:30:11.744234
RANDOMNESS =  0.2
BoardName.RANDOM_SEED
100 Evaluations: Average connections for random_seed = 3.92, std=0.9019977827023744
2023-04-27 12:30:40.660943
RANDOMNESS =  0.4
BoardName.RANDOM_SEED
100 Evaluations: Average connections for random_seed = 3.92, std=0.8681013765684281
2023-04-27 12:31:08.471889
RANDOMNESS =  0.6000000000000001
BoardName.RANDOM_SEED
100 Evaluations: Average connections for random_seed = 4.01, std=0.9433451118228154
2023-04-27 12:31:37.653821
RANDOMNESS =  0.8
BoardName.RANDOM_SEED
100 Evaluations: Average connections for random_seed = 3.84, std=0.9562426470305537
2023-04-27 12:32:06.387890
100 Evaluations: Total average connections for uniform = 3.952, std=0.9086781608468425
CORRELATIONS
mean keys =  2123742063.32
mean episodes =  49.5
Correlation of connections with detours is  0.0006230377283124016
Correlation of connections with keys is  -0.04040291085846616
Correlation of connections with episodes is  -0.03225318528800453
Correlation of connections with densities is  0.10042128970381678

AGENT =  bfs_short
2023-04-27 12:32:06.534606
RANDOMNESS =  0.0
BoardName.RANDOM_SEED
100 Evaluations: Average connections for random_seed = 2.4, std=1.3114877048604001
2023-04-27 12:32:38.462151
RANDOMNESS =  0.2
BoardName.RANDOM_SEED
100 Evaluations: Average connections for random_seed = 2.35, std=1.2114041439585717
2023-04-27 12:33:08.766875
RANDOMNESS =  0.4
BoardName.RANDOM_SEED
100 Evaluations: Average connections for random_seed = 2.21, std=1.168717245530329
2023-04-27 12:33:40.693133
RANDOMNESS =  0.6000000000000001
BoardName.RANDOM_SEED
100 Evaluations: Average connections for random_seed = 2.29, std=1.1161989070053777
2023-04-27 12:34:10.698548
RANDOMNESS =  0.8
BoardName.RANDOM_SEED
100 Evaluations: Average connections for random_seed = 2.44, std=1.2026637102698328
2023-04-27 12:34:41.648861
100 Evaluations: Total average connections for bfs_short = 2.338, std=1.2065471395681149
CORRELATIONS
mean keys =  2071304389.074
mean episodes =  49.5
Correlation of connections with detours is  0.040127154735123516
Correlation of connections with keys is  0.029243358684356787
Correlation of connections with episodes is  -0.027994496456962028
Correlation of connections with densities is  -0.06355071714367526

AGENT =  random_walk
2023-04-27 12:34:41.781718
RANDOMNESS =  0.0
BoardName.RANDOM_SEED
100 Evaluations: Average connections for random_seed = 3.74, std=1.0547037498748169
2023-04-27 12:35:10.987303
RANDOMNESS =  0.2
BoardName.RANDOM_SEED
100 Evaluations: Average connections for random_seed = 3.63, std=0.9965440281292143
2023-04-27 12:35:40.425833
RANDOMNESS =  0.4
BoardName.RANDOM_SEED
100 Evaluations: Average connections for random_seed = 3.66, std=1.1065260954898444
2023-04-27 12:36:08.759072
RANDOMNESS =  0.6000000000000001
BoardName.RANDOM_SEED
100 Evaluations: Average connections for random_seed = 3.6, std=1.1661903789690602
2023-04-27 12:36:38.004551
RANDOMNESS =  0.8
BoardName.RANDOM_SEED
100 Evaluations: Average connections for random_seed = 3.39, std=1.139254141971843
2023-04-27 12:37:07.445663
100 Evaluations: Total average connections for random_walk = 3.604, std=1.1005380502281599
CORRELATIONS
mean keys =  2069023301.978
mean episodes =  49.5
Correlation of connections with detours is  -0.0014610346541959686
Correlation of connections with keys is  0.08742786169773888
Correlation of connections with episodes is  -0.009380447137493777
Correlation of connections with densities is  0.10457479995457367

AGENT =  numberlink
2023-04-27 12:37:07.577889
RANDOMNESS =  0.0
BoardName.RANDOM_SEED
100 Evaluations: Average connections for random_seed = 3.7, std=0.9433981132056604
2023-04-27 12:37:37.328156
RANDOMNESS =  0.2
BoardName.RANDOM_SEED
100 Evaluations: Average connections for random_seed = 3.66, std=1.060377291344925
2023-04-27 12:38:07.288507
RANDOMNESS =  0.4
BoardName.RANDOM_SEED
100 Evaluations: Average connections for random_seed = 3.68, std=1.0571660229122009
2023-04-27 12:38:36.370159
RANDOMNESS =  0.6000000000000001
BoardName.RANDOM_SEED
100 Evaluations: Average connections for random_seed = 3.59, std=1.0871522432483869
2023-04-27 12:39:06.315704
RANDOMNESS =  0.8
BoardName.RANDOM_SEED
100 Evaluations: Average connections for random_seed = 3.42, std=1.159137610467368
2023-04-27 12:39:35.744557
100 Evaluations: Total average connections for numberlink = 3.61, std=1.068597211300872
CORRELATIONS
mean keys =  2156502997.742
mean episodes =  49.5
Correlation of connections with detours is  0.00822071052421733
Correlation of connections with keys is  -0.1007668153963554
Correlation of connections with episodes is  -0.054755525575147776
Correlation of connections with densities is  0.11485135491929124

AGENT =  lsystems
2023-04-27 12:39:35.880196
RANDOMNESS =  0.0
BoardName.RANDOM_SEED
100 Evaluations: Average connections for random_seed = 2.57, std=1.2186467905016614
2023-04-27 12:40:09.339157
RANDOMNESS =  0.2
BoardName.RANDOM_SEED
100 Evaluations: Average connections for random_seed = 2.53, std=1.2605950975630518
2023-04-27 12:40:40.465252
RANDOMNESS =  0.4
BoardName.RANDOM_SEED
100 Evaluations: Average connections for random_seed = 2.67, std=1.2004582458378135
2023-04-27 12:41:13.681859
RANDOMNESS =  0.6000000000000001
BoardName.RANDOM_SEED
100 Evaluations: Average connections for random_seed = 2.53, std=1.1353853971229328
2023-04-27 12:41:46.208440
RANDOMNESS =  0.8
BoardName.RANDOM_SEED
100 Evaluations: Average connections for random_seed = 2.54, std=1.2443472184241824
2023-04-27 12:42:19.149017
100 Evaluations: Total average connections for lsystems = 2.568, std=1.213827005796131
CORRELATIONS
mean keys =  2140709598.054
mean episodes =  49.5
Correlation of connections with detours is  0.11267256895514584
Correlation of connections with keys is  -0.022635759568808193
Correlation of connections with episodes is  0.03967074381413648
Correlation of connections with densities is  0.0359749595882062


NUMPY AGENTS ON ALL THE BOARDS WITH CORRELATIONS

AGENT =  uniform
2023-04-27 01:38:02.818085
BoardName.BFS_BASE
5000 Evaluations: Average connections for bfs_base = 4.5526, std=0.6213157329409904
2023-04-27 01:44:39.061435
BoardName.BFS_SHORTEST
5000 Evaluations: Average connections for bfs_short = 4.433, std=0.6704558150989519
2023-04-27 01:52:13.106151
BoardName.BFS_LONGEST
5000 Evaluations: Average connections for bfs_long = 4.715, std=0.5217039390305578
2023-04-27 01:59:09.432143
BoardName.RANDOM_WALK
5000 Evaluations: Average connections for random_walk = 4.6748, std=0.5428120853481433
2023-04-27 02:05:33.613297
BoardName.RANDOM_SEED

/usr/local/lib/python3.10/dist-packages/jax/_src/ops/scatter.py:89: FutureWarning: scatter inputs have incompatible types: cannot safely cast value from dtype=float32 to dtype=int32. In future JAX releases this will result in an error.
  warnings.warn("scatter inputs have incompatible types: cannot safely cast "

5000 Evaluations: Average connections for random_seed = 3.8484, std=0.9515342558205668
2023-04-27 02:13:29.178871
BoardName.NUMBERLINK
5000 Evaluations: Average connections for numberlink = 4.0126, std=0.9031285844219525
2023-04-27 02:26:10.815254
5000 Evaluations: Total average connections for uniform = 4.372733333333334, std=0.7929290818786312
CORRELATIONS
mean keys =  2141478808.8718333
mean episodes =  2499.5
Correlation of connections with detours is  -0.3043430472800306
Correlation of connections with keys is  0.0021147426817292522
Correlation of connections with episodes is  0.005312942267783601
Correlation of connections with densities is  -0.3609169004267374

AGENT =  bfs_short
2023-04-27 02:26:12.624943
BoardName.BFS_BASE
5000 Evaluations: Average connections for bfs_base = 2.9382, std=1.1651526766909133
2023-04-27 02:34:10.730764
BoardName.BFS_SHORTEST
5000 Evaluations: Average connections for bfs_short = 2.8004, std=1.1554046217667646
2023-04-27 02:42:41.053300
BoardName.BFS_LONGEST
5000 Evaluations: Average connections for bfs_long = 3.1554, std=1.1532783011918677
2023-04-27 02:51:07.922740
BoardName.RANDOM_WALK
5000 Evaluations: Average connections for random_walk = 3.0934, std=1.1728070770591386
2023-04-27 02:58:52.480920
BoardName.RANDOM_SEED
5000 Evaluations: Average connections for random_seed = 2.3412, std=1.1874268651163322
2023-04-27 03:08:00.446338
BoardName.NUMBERLINK
5000 Evaluations: Average connections for numberlink = 3.8424, std=0.9438020131362297
2023-04-27 03:20:53.089756
5000 Evaluations: Total average connections for bfs_short = 3.0285, std=1.2188879152735905
CORRELATIONS
mean keys =  2158922482.734667
mean episodes =  2499.5
Correlation of connections with detours is  0.14772532102739205
Correlation of connections with keys is  0.00345210321304468
Correlation of connections with episodes is  0.0036621962803893688
Correlation of connections with densities is  0.07403909554748482

AGENT =  random_walk
2023-04-27 03:20:54.938323
BoardName.BFS_BASE
5000 Evaluations: Average connections for bfs_base = 4.2936, std=0.8360616245229774
2023-04-27 03:28:02.523034
BoardName.BFS_SHORTEST
5000 Evaluations: Average connections for bfs_short = 4.153, std=0.9024361473256709
2023-04-27 03:35:59.995396
BoardName.BFS_LONGEST
5000 Evaluations: Average connections for bfs_long = 4.5598, std=0.6797234437622406
2023-04-27 03:43:27.248951
BoardName.RANDOM_WALK
5000 Evaluations: Average connections for random_walk = 4.5262, std=0.6940558767131073
2023-04-27 03:50:18.612485
BoardName.RANDOM_SEED
5000 Evaluations: Average connections for random_seed = 3.488, std=1.1373020706918633
2023-04-27 03:59:04.850276
BoardName.NUMBERLINK
5000 Evaluations: Average connections for numberlink = 3.6562, std=1.1000007090906805
2023-04-27 04:12:25.304601
5000 Evaluations: Total average connections for random_walk = 4.1128, std=0.9970336804742355
CORRELATIONS
mean keys =  2150297489.4328
mean episodes =  2499.5
Correlation of connections with detours is  -0.30260609388110443
Correlation of connections with keys is  -0.003455071061657189
Correlation of connections with episodes is  -0.005974614967576746
Correlation of connections with densities is  -0.35947118869083433

AGENT =  numberlink
2023-04-27 04:12:27.154864
BoardName.BFS_BASE
5000 Evaluations: Average connections for bfs_base = 4.3752, std=0.7715082371562859
2023-04-27 04:19:50.570324
BoardName.BFS_SHORTEST
5000 Evaluations: Average connections for bfs_short = 4.2092, std=0.8517249321230418
2023-04-27 04:28:03.044013
BoardName.BFS_LONGEST
5000 Evaluations: Average connections for bfs_long = 4.5708, std=0.6586253563293779
2023-04-27 04:35:41.012380
BoardName.RANDOM_WALK
5000 Evaluations: Average connections for random_walk = 4.46, std=0.728285658241325
2023-04-27 04:42:49.485739
BoardName.RANDOM_SEED
5000 Evaluations: Average connections for random_seed = 3.4458, std=1.1111536167425278
2023-04-27 04:51:57.277979
BoardName.NUMBERLINK
5000 Evaluations: Average connections for numberlink = 3.6776, std=1.056247243783386
2023-04-27 05:05:17.503815
5000 Evaluations: Total average connections for numberlink = 4.1231, std=0.9727348336862758
CORRELATIONS
mean keys =  2144390277.6921
mean episodes =  2499.5
Correlation of connections with detours is  -0.3225426422820274
Correlation of connections with keys is  -0.0017784807648642718
Correlation of connections with episodes is  -0.00047481457596557227
Correlation of connections with densities is  -0.3783451306493704

AGENT =  lsystems
2023-04-27 05:05:19.325002
BoardName.BFS_BASE
5000 Evaluations: Average connections for bfs_base = 3.2328, std=1.096815463056571
2023-04-27 05:13:52.160406
BoardName.BFS_SHORTEST
5000 Evaluations: Average connections for bfs_short = 2.972, std=1.098187597817422
2023-04-27 05:23:05.029783
BoardName.BFS_LONGEST
5000 Evaluations: Average connections for bfs_long = 3.676, std=1.026948878961363
2023-04-27 05:31:45.091908
BoardName.RANDOM_WALK
5000 Evaluations: Average connections for random_walk = 3.5324, std=1.039879916144167
2023-04-27 05:40:03.446062
BoardName.RANDOM_SEED
5000 Evaluations: Average connections for random_seed = 2.4232, std=1.169658822050259
2023-04-27 05:49:29.028068
BoardName.NUMBERLINK
5000 Evaluations: Average connections for numberlink = 2.7394, std=1.1523400713331113
2023-04-27 06:03:45.221117
5000 Evaluations: Total average connections for lsystems = 3.0959666666666665, std=1.181929382643293
CORRELATIONS
mean keys =  2141372274.1216333
mean episodes =  2499.5
Correlation of connections with detours is  -0.224315057132908
Correlation of connections with keys is  0.0005751917077445454
Correlation of connections with episodes is  -0.004305912911889493
Correlation of connections with densities is  -0.2977234082438335



AGENT =  uniform
2023-04-27 00:51:14.738551
BoardName.BFS_BASE
2000 Evaluations: Average connections for bfs_base = 4.5565, std=0.6227421215880615
2023-04-27 00:54:05.485229
BoardName.BFS_SHORTEST
2000 Evaluations: Average connections for bfs_short = 4.4545, std=0.6737430890183588
2023-04-27 00:57:14.682526
BoardName.BFS_LONGEST
2000 Evaluations: Average connections for bfs_long = 4.704, std=0.521904205769603
2023-04-27 01:00:11.413419
BoardName.RANDOM_WALK
2000 Evaluations: Average connections for random_walk = 4.655, std=0.5347663040992766
2023-04-27 01:02:53.516845
BoardName.RANDOM_SEED
2000 Evaluations: Average connections for random_seed = 3.8195, std=0.9412331007779103
2023-04-27 01:06:12.327947
BoardName.NUMBERLINK
2000 Evaluations: Average connections for numberlink = 4.025, std=0.9051933495115836
2023-04-27 01:11:26.095806
2000 Evaluations: Total average connections for uniform = 4.369083333333333, std=0.7918506759835187
CORRELATIONS
mean keys =  2151179503.337
mean episodes =  999.5
Correlation of connections with detours is  -0.3039394602550464
Correlation of connections with keys is  -0.010371968415504144
Correlation of connections with episodes is  -0.013039951422649666
Correlation of connections with densities is  -0.3642894264295563

AGENT =  bfs_short
2023-04-27 01:11:28.166212
BoardName.BFS_BASE
2000 Evaluations: Average connections for bfs_base = 2.9585, std=1.1767658008286952
2023-04-27 01:14:47.390867
BoardName.BFS_SHORTEST
2000 Evaluations: Average connections for bfs_short = 2.826, std=1.173764882759746
2023-04-27 01:18:20.636179
BoardName.BFS_LONGEST
2000 Evaluations: Average connections for bfs_long = 3.1795, std=1.1829115562881276
2023-04-27 01:21:49.808128
BoardName.RANDOM_WALK
2000 Evaluations: Average connections for random_walk = 3.096, std=1.1746420731439855
2023-04-27 01:25:01.520556
BoardName.RANDOM_SEED
2000 Evaluations: Average connections for random_seed = 2.1625, std=1.134060734705157
2023-04-27 01:28:48.544430
BoardName.NUMBERLINK
2000 Evaluations: Average connections for numberlink = 3.828, std=0.981537569326819
2023-04-27 01:34:16.742333
2000 Evaluations: Total average connections for bfs_short = 3.0084166666666667, std=1.2417108465294522
CORRELATIONS
mean keys =  2150255903.085
mean episodes =  999.5
Correlation of connections with detours is  0.12070753195707394
Correlation of connections with keys is  -0.013698554104696786
Correlation of connections with episodes is  -0.01668016219120098
Correlation of connections with densities is  0.03733892084608622

AGENT =  random_walk
2023-04-27 01:34:17.574740
BoardName.BFS_BASE


In [ ]:
EVALUATED AGENTS TRAINED ON RANDOMSEED


AGENTS TRAINED WITH INFINITE-STEPS AND RANDOMNESS=1, VARYING ITERATIONS
AGENT =  iterations1
2023-04-26 13:31:02.824958
BoardName.BFS_BASE
2000 Evaluations: Average connections for bfs_base = 1.119, std=0.9401271190642252
2023-04-26 13:34:30.998542
BoardName.BFS_SHORTEST
2000 Evaluations: Average connections for bfs_short = 0.9915, std=0.9232701392333664
2023-04-26 13:37:58.994153
BoardName.BFS_LONGEST
2000 Evaluations: Average connections for bfs_long = 1.3275, std=1.0350090579313787
2023-04-26 13:41:25.754858
BoardName.RANDOM_WALK
2000 Evaluations: Average connections for random_walk = 1.3115, std=1.0456900831508351
2023-04-26 13:44:46.164472
BoardName.RANDOM_SEED
2000 Evaluations: Average connections for random_seed = 1.3715, std=1.0571129315262395
2023-04-26 13:48:20.192809
BoardName.NUMBERLINK
2000 Evaluations: Average connections for numberlink = 0.866, std=0.8532549443161755
2023-04-26 13:53:58.876942
2000 Evaluations: Total average connections for iterations1 = 1.1645, std=0.9965472475837093

AGENT =  iterations2
2023-04-26 13:53:59.113240
BoardName.BFS_BASE
2000 Evaluations: Average connections for bfs_base = 1.112, std=0.9340535316565106
2023-04-26 13:57:20.927835
BoardName.BFS_SHORTEST
2000 Evaluations: Average connections for bfs_short = 0.9415, std=0.8843515986303185
2023-04-26 14:00:50.294252
BoardName.BFS_LONGEST
2000 Evaluations: Average connections for bfs_long = 1.247, std=1.003987549723601
2023-04-26 14:04:23.400428
BoardName.RANDOM_WALK
2000 Evaluations: Average connections for random_walk = 1.244, std=1.0116639758338735
2023-04-26 14:07:47.002479
BoardName.RANDOM_SEED
2000 Evaluations: Average connections for random_seed = 1.481, std=1.0773295688878126
2023-04-26 14:11:26.783384
BoardName.NUMBERLINK
2000 Evaluations: Average connections for numberlink = 0.544, std=0.7007595878758991
2023-04-26 14:17:14.841876
2000 Evaluations: Total average connections for iterations2 = 1.0949166666666668, std=0.9882176681896667

AGENT =  iterations5
2023-04-26 14:17:15.077602
BoardName.BFS_BASE
2000 Evaluations: Average connections for bfs_base = 1.091, std=0.9548397771354104
2023-04-26 14:20:39.352295
BoardName.BFS_SHORTEST
2000 Evaluations: Average connections for bfs_short = 0.9255, std=0.880312302538139
2023-04-26 14:24:07.499759
BoardName.BFS_LONGEST
2000 Evaluations: Average connections for bfs_long = 1.2975, std=1.0044868092712815
2023-04-26 14:27:39.386712
BoardName.RANDOM_WALK
2000 Evaluations: Average connections for random_walk = 1.2345, std=1.001254088630853
2023-04-26 14:31:03.072436
BoardName.RANDOM_SEED
2000 Evaluations: Average connections for random_seed = 1.31, std=1.0741973747873341
2023-04-26 14:34:40.878467
BoardName.NUMBERLINK
2000 Evaluations: Average connections for numberlink = 0.777, std=0.8113390166878455
2023-04-26 14:40:20.602238
2000 Evaluations: Total average connections for iterations5 = 1.1059166666666667, std=0.9786206243426964

AGENT =  iterations6
2023-04-26 14:40:20.836595
BoardName.BFS_BASE
2000 Evaluations: Average connections for bfs_base = 1.1925, std=0.9856184606631513
2023-04-26 14:43:51.937758
BoardName.BFS_SHORTEST
2000 Evaluations: Average connections for bfs_short = 1.1245, std=0.9370164086076616
2023-04-26 14:47:30.359968
BoardName.BFS_LONGEST
2000 Evaluations: Average connections for bfs_long = 1.367, std=1.0184846586964382
2023-04-26 14:51:09.893074
BoardName.RANDOM_WALK
2000 Evaluations: Average connections for random_walk = 1.322, std=1.0179960707193323
2023-04-26 14:55:03.477294
BoardName.RANDOM_SEED
2000 Evaluations: Average connections for random_seed = 1.2955, std=1.0378727041405413
2023-04-26 14:59:31.432176
BoardName.NUMBERLINK
2000 Evaluations: Average connections for numberlink = 1.351, std=0.9347721647545995
2023-04-26 15:06:11.125980
2000 Evaluations: Total average connections for iterations6 = 1.2754166666666666, std=0.9933423342713003

AGENT =  iterations8
2023-04-26 15:06:11.380268
BoardName.BFS_BASE
2000 Evaluations: Average connections for bfs_base = 1.0355, std=0.9398083581241443
2023-04-26 15:10:04.032445
BoardName.BFS_SHORTEST
2000 Evaluations: Average connections for bfs_short = 0.9725, std=0.8987456536751652
2023-04-26 15:14:00.171128
BoardName.BFS_LONGEST
2000 Evaluations: Average connections for bfs_long = 1.215, std=0.9908455984662797
2023-04-26 15:17:53.129872
BoardName.RANDOM_WALK
2000 Evaluations: Average connections for random_walk = 1.266, std=1.0006218066782273
2023-04-26 15:21:51.414278
BoardName.RANDOM_SEED
2000 Evaluations: Average connections for random_seed = 0.997, std=0.9433933432031414
2023-04-26 15:26:15.827504
BoardName.NUMBERLINK
2000 Evaluations: Average connections for numberlink = 1.0905, std=0.9794435920460146
2023-04-26 15:32:43.474826
2000 Evaluations: Total average connections for iterations8 = 1.0960833333333333, std=0.9656697121974757

In [ ]:
COMPARING TIMES WITH AND WITHOUT BFS OPTIMISATION (1 ITERATION OF EXTENSION)

WITH EXTENSION =  7MIN 38SEC, 7min 21sec

AGENT =  uniform
2023-04-26 11:41:18.443837
BoardName.RANDOM_SEED
5000 Evaluations: Average connections for random_seed = 4.4606, std=0.692565982416116
2023-04-26 11:48:56.098630
        
AGENT =  uniform
2023-04-26 11:52:39.877301
BoardName.RANDOM_SEED
5000 Evaluations: Average connections for random_seed = 4.4606, std=0.692565982416116
2023-04-26 12:00:10.555290



WITHOUT EXTENSION = 6 MIN 47 SEC

AGENT =  uniform
2023-04-26 10:35:12.227726
BoardName.RANDOM_SEED
5000 Evaluations: Average connections for random_seed = 4.4606, std=0.692565982416116
2023-04-26 10:41:59.222164




TEST RESULTS OF NUMPY AGENTS ON (ALMOST) ALL BOARDS
AGENT =  uniform
2023-04-25 23:47:31.936717
BoardName.BFS_BASE
5000 Evaluations: Average connections for bfs_base = 4.5616, std=0.6156341770889592
2023-04-25 23:54:36.309006
BoardName.BFS_MIN_BENDS
5000 Evaluations: Average connections for bfs_min_bend = 4.5048, std=0.6548106291134865
2023-04-26 00:02:09.791877
BoardName.BFS_SHORTEST
5000 Evaluations: Average connections for bfs_short = 4.4262, std=0.6906182447633424
2023-04-26 00:09:17.828194
BoardName.BFS_LONGEST
5000 Evaluations: Average connections for bfs_long = 4.7214, std=0.5061442877283118
2023-04-26 00:15:49.293854
BoardName.RANDOM_WALK
5000 Evaluations: Average connections for random_walk = 4.6738, std=0.5427647372481009
2023-04-26 00:22:15.042061
BoardName.RANDOM_SEED
5000 Evaluations: Average connections for random_seed = 3.8534, std=0.9589100270619763
2023-04-26 00:30:11.145025
BoardName.WFC
5000 Evaluations: Average connections for wfc = 4.8294, std=0.4141203206798719
2023-04-26 01:02:28.658679
BoardName.NUMBERLINK
5000 Evaluations: Average connections for numberlink = 4.0318, std=0.904206149061153
2023-04-26 01:15:28.361175

AGENT =  bfs_short
2023-04-26 01:15:28.466341
BoardName.BFS_BASE
5000 Evaluations: Average connections for bfs_base = 2.939, std=1.1677666718998276
2023-04-26 01:23:25.850473
BoardName.BFS_MIN_BENDS
5000 Evaluations: Average connections for bfs_min_bend = 2.8648, std=1.1696670295430234
2023-04-26 01:31:55.247815
BoardName.BFS_SHORTEST
5000 Evaluations: Average connections for bfs_short = 2.8002, std=1.1627037283848367
2023-04-26 01:40:12.851809
BoardName.BFS_LONGEST
5000 Evaluations: Average connections for bfs_long = 3.1776, std=1.1627803919915403
2023-04-26 01:48:25.498068
BoardName.RANDOM_WALK
5000 Evaluations: Average connections for random_walk = 3.0686, std=1.162193632748003
2023-04-26 01:56:24.193549
BoardName.RANDOM_SEED
5000 Evaluations: Average connections for random_seed = 2.1476, std=1.1186662773141953
2023-04-26 02:05:27.206038
BoardName.WFC
5000 Evaluations: Average connections for wfc = 4.5914, std=0.6440854291163556
2023-04-26 02:38:20.015540
BoardName.NUMBERLINK
5000 Evaluations: Average connections for numberlink = 3.8352, std=0.965215499253923
2023-04-26 02:51:39.969372

AGENT =  random_walk
2023-04-26 02:51:40.081084
BoardName.BFS_BASE
5000 Evaluations: Average connections for bfs_base = 4.3218, std=0.8172176943752503
2023-04-26 02:59:02.256643
BoardName.BFS_MIN_BENDS
5000 Evaluations: Average connections for bfs_min_bend = 4.2388, std=0.8577730235907398
2023-04-26 03:06:56.172927
BoardName.BFS_SHORTEST
5000 Evaluations: Average connections for bfs_short = 4.1624, std=0.8768273718355285
2023-04-26 03:14:58.474362
BoardName.BFS_LONGEST
5000 Evaluations: Average connections for bfs_long = 4.5598, std=0.6803116638717874
2023-04-26 03:22:21.284912
BoardName.RANDOM_WALK
5000 Evaluations: Average connections for random_walk = 4.5128, std=0.709814172864983
2023-04-26 03:29:39.383039
BoardName.RANDOM_SEED
5000 Evaluations: Average connections for random_seed = 3.4714, std=1.1265797974400216
2023-04-26 03:39:01.862322
BoardName.WFC
5000 Evaluations: Average connections for wfc = 4.7724, std=0.5001982007164759
2023-04-26 04:11:57.579422
BoardName.NUMBERLINK
5000 Evaluations: Average connections for numberlink = 3.644, std=1.109983783665329
2023-04-26 04:26:05.278556

AGENT =  numberlink
2023-04-26 04:26:05.399620
BoardName.BFS_BASE
5000 Evaluations: Average connections for bfs_base = 4.3372, std=0.7991846845379359
2023-04-26 04:33:49.242904
BoardName.BFS_MIN_BENDS
5000 Evaluations: Average connections for bfs_min_bend = 4.2562, std=0.8235056526824817
2023-04-26 04:42:13.055582
BoardName.BFS_SHORTEST
5000 Evaluations: Average connections for bfs_short = 4.2214, std=0.849695263020808
2023-04-26 04:50:33.067931
BoardName.BFS_LONGEST
5000 Evaluations: Average connections for bfs_long = 4.573, std=0.6656357862975818
2023-04-26 04:58:10.367446
BoardName.RANDOM_WALK
5000 Evaluations: Average connections for random_walk = 4.483, std=0.727812475848003
2023-04-26 05:05:50.925140
BoardName.RANDOM_SEED
5000 Evaluations: Average connections for random_seed = 3.4868, std=1.0906079772310489
2023-04-26 05:15:11.024084
BoardName.WFC
5000 Evaluations: Average connections for wfc = 4.7162, std=0.5444791639723232
2023-04-26 05:47:51.963054
BoardName.NUMBERLINK
5000 Evaluations: Average connections for numberlink = 3.6354, std=1.064925743890155
2023-04-26 06:01:37.529204

AGENT =  lsystems
2023-04-26 06:01:37.636882
BoardName.BFS_BASE
5000 Evaluations: Average connections for bfs_base = 3.2256, std=1.0895433171746776
2023-04-26 06:10:11.088043
BoardName.BFS_MIN_BENDS
5000 Evaluations: Average connections for bfs_min_bend = 3.1334, std=1.116603976349717
2023-04-26 06:18:48.654105
BoardName.BFS_SHORTEST
5000 Evaluations: Average connections for bfs_short = 2.9472, std=1.1124801840931822
2023-04-26 06:27:24.561674
BoardName.BFS_LONGEST
5000 Evaluations: Average connections for bfs_long = 3.6326, std=1.0486263586235087
2023-04-26 06:35:36.462384
BoardName.RANDOM_WALK
5000 Evaluations: Average connections for random_walk = 3.5122, std=1.0462557813460338
2023-04-26 06:43:48.704323
BoardName.RANDOM_SEED
5000 Evaluations: Average connections for random_seed = 2.4956, std=1.1701199254777264
2023-04-26 06:53:02.639827
BoardName.WFC
5000 Evaluations: Average connections for wfc = 3.9088, std=0.9796338907979858
2023-04-26 07:27:07.950632
BoardName.NUMBERLINK
5000 Evaluations: Average connections for numberlink = 2.7446, std=1.1298543445949127
2023-04-26 07:41:45.950318


RANDOMSEED BOARD TIME ESTIMATES

W/O RUNNING AGENTS
10000 BOARDS (21 ITERATIONS, INFINITE WIRE LENGTH) IN ABOUT 1.5 MINUTES 
.01 SEC/BOARD TO GENERATE

W/ AGENTS
1 BOARD PRECOMPILED = 7 SEC
2023-04-25 14:07:12.295666
1 Evaluations: Average connections for random_seed = 4.0, std=0.0
2023-04-25 14:07:19.009992

10 BOARDS = 7 sec
2023-04-25 14:10:05.607338
10 Evaluations: Average connections for random_seed = 3.4, std=1.1135528725660044
2023-04-25 14:10:12.976135

100 BOARDS = 16 SEC
2023-04-25 14:11:07.214522
2023-04-25 14:11:23.295869

1000 BOARDS = 1 MIN 36 sec = 0.1 sec/board to solve
2023-04-25 14:12:27.030828
1000 Evaluations: Average connections for random_seed = 3.909, std=0.9288266792033916
2023-04-25 14:14:03.844435





1 BOARD, PRE-COMPILED

2023-04-25 13:14:16.051197
BoardName.RANDOM_SEED
SEEDED BOARD
Traced<ShapedArray(int32[10,10])>with<DynamicJaxprTrace(level=1/0)>
Extension iteration  Traced<ShapedArray(int32[], weak_type=True)>with<DynamicJaxprTrace(level=2/0)>
Extension Step  Traced<ShapedArray(int32[], weak_type=True)>with<DynamicJaxprTrace(level=3/0)>
Traced<ShapedArray(int32[10,10])>with<DynamicJaxprTrace(level=2/0)>
Optimization
Traced<ShapedArray(int32[10,10])>with<DynamicJaxprTrace(level=2/0)>
1 Evaluations: Average connections for random_seed = 4.0, std=0.0
2023-04-25 13:14:22.602786


7 ITERATIONS, COMPILATION TIME STILL AT 16, CONNECTIONS BARELY DOWN TO 3.87

2023-04-25 08:03:44.757613
BoardName.RANDOM_SEED
SEEDED BOARD
Traced<ShapedArray(int32[10,10])>with<DynamicJaxprTrace(level=1/0)>
Extension iteration  Traced<ShapedArray(int32[], weak_type=True)>with<DynamicJaxprTrace(level=2/0)>
Extension Step  Traced<ShapedArray(int32[], weak_type=True)>with<DynamicJaxprTrace(level=3/0)>
Traced<ShapedArray(int32[10,10])>with<DynamicJaxprTrace(level=2/0)>
Optimization
Traced<ShapedArray(int32[10,10])>with<DynamicJaxprTrace(level=2/0)>
10000 Evaluations: Average connections for random_seed = 3.8715, std=0.930799521916508
2023-04-25 08:19:40.966279



DECLARING ALL VARIABLES STATIC HAD NO EFFECT
5 ITERATIONS, COMPILATION TIME UP TO 16, CONNECTIONS DOWN TO 3.9

2023-04-25 07:37:21.125275
BoardName.RANDOM_SEED
SEEDED BOARD
Traced<ShapedArray(int32[10,10])>with<DynamicJaxprTrace(level=1/0)>
Extension iteration  Traced<ShapedArray(int32[], weak_type=True)>with<DynamicJaxprTrace(level=2/0)>
Extension Step  Traced<ShapedArray(int32[], weak_type=True)>with<DynamicJaxprTrace(level=3/0)>
Traced<ShapedArray(int32[10,10])>with<DynamicJaxprTrace(level=2/0)>
Optimization
Traced<ShapedArray(int32[10,10])>with<DynamicJaxprTrace(level=2/0)>
10000 Evaluations: Average connections for random_seed = 3.9009, std=0.9294510153848885
2023-04-25 07:53:02.718436







5 ITERATIONS, COMPILATION TIME UP TO 16, CONNECTIONS DOWN TO 3.9
2023-04-25 00:40:19.011315
BoardName.RANDOM_SEED
SEEDED BOARD
Traced<ShapedArray(int32[10,10])>with<DynamicJaxprTrace(level=1/0)>
Extension iteration  Traced<ShapedArray(int32[], weak_type=True)>with<DynamicJaxprTrace(level=2/0)>
Extension Step  Traced<ShapedArray(int32[], weak_type=True)>with<DynamicJaxprTrace(level=3/0)>
Traced<ShapedArray(int32[10,10])>with<DynamicJaxprTrace(level=2/0)>
Optimization
Traced<ShapedArray(int32[10,10])>with<DynamicJaxprTrace(level=2/0)>
10000 Evaluations: Average connections for random_seed = 3.9009, std=0.9294510153848885
2023-04-25 00:56:22.599802





AFTER SLIGHT REFINEMENT, COMPILATION TIME FOR 3 ITERATIONS IS ~14 MINUTES
2023-04-25 00:12:48.872868
BoardName.RANDOM_SEED
SEEDED BOARD
Traced<ShapedArray(int32[10,10])>with<DynamicJaxprTrace(level=1/0)>
Extension iteration  Traced<ShapedArray(int32[], weak_type=True)>with<DynamicJaxprTrace(level=2/0)>
Extension Step  Traced<ShapedArray(int32[], weak_type=True)>with<DynamicJaxprTrace(level=3/0)>
Traced<ShapedArray(int32[10,10])>with<DynamicJaxprTrace(level=2/0)>
Optimization
Traced<ShapedArray(int32[10,10])>with<DynamicJaxprTrace(level=2/0)>

/usr/local/lib/python3.10/dist-packages/jax/_src/ops/scatter.py:89: FutureWarning: scatter inputs have incompatible types: cannot safely cast value from dtype=float32 to dtype=int32. In future JAX releases this will result in an error.
  warnings.warn("scatter inputs have incompatible types: cannot safely cast "

10000 Evaluations: Average connections for random_seed = 4.0198, std=0.888823919570125
2023-04-25 00:26:39.313537







COMPILATION TIME WITH NUM_ITERATIONS =1 IS ABOUT THE SAME AS =2

2023-04-24 22:04:12.410881
BoardName.RANDOM_SEED
SEEDED BOARD
Traced<ShapedArray(int32[10,10])>with<DynamicJaxprTrace(level=1/0)>
Extension iteration  Traced<ShapedArray(int32[], weak_type=True)>with<DynamicJaxprTrace(level=2/0)>
Extension Step  Traced<ShapedArray(int32[], weak_type=True)>with<DynamicJaxprTrace(level=3/0)>
Traced<ShapedArray(int32[10,10])>with<DynamicJaxprTrace(level=2/0)>
Optimization
Traced<ShapedArray(int32[10,10])>with<DynamicJaxprTrace(level=2/0)>
10000 Evaluations: Average connections for random_seed = 4.451, std=0.7166582170044519
2023-04-24 22:19:36.665689




COMPILE 
solved_board = solved_board_compiled(subkey, randomness = 1.0, two_sided = True,extension_iterations=int(2), max_extension=1e23)
2023-04-24 14:08:33.452352
BoardName.RANDOM_SEED
10000 Evaluations: Average connections for random_seed = 4.1619, std=0.8239468368772346
2023-04-24 14:23:46.241724






COMPILE WITH UGO'S FIX TO MAKE THE BFS OPTIMIZER JITABLE
2023-04-24 13:35:57.047568
BoardName.RANDOM_SEED
SEEDED BOARD
Traced<ShapedArray(int32[10,10])>with<DynamicJaxprTrace(level=1/0)>
type(extension_iterations) = <class 'int'>
Extension iteration  1
Extension Step  Traced<ShapedArray(int32[], weak_type=True)>with<DynamicJaxprTrace(level=2/0)>
Traced<ShapedArray(int32[10,10])>with<DynamicJaxprTrace(level=1/0)>

/usr/local/lib/python3.10/dist-packages/jax/_src/ops/scatter.py:89: FutureWarning: scatter inputs have incompatible types: cannot safely cast value from dtype=float32 to dtype=int32. In future JAX releases this will result in an error.
  warnings.warn("scatter inputs have incompatible types: cannot safely cast "

Optimization
Traced<ShapedArray(int32[10,10])>with<DynamicJaxprTrace(level=1/0)>
Extension iteration  2
Extension Step  Traced<ShapedArray(int32[], weak_type=True)>with<DynamicJaxprTrace(level=2/0)>
Traced<ShapedArray(int32[10,10])>with<DynamicJaxprTrace(level=1/0)>
Optimization
Traced<ShapedArray(int32[10,10])>with<DynamicJaxprTrace(level=1/0)>
10000 Evaluations: Average connections for random_seed = 4.1619, std=0.8239468368772346
2023-04-24 13:49:36.908408






COMPILE TIME AFTER REMOVING A COUPLE FOR LOOPS = 13MIN 24 SEC
2023-04-23 13:25:04.396975
BoardName.RANDOM_SEED
10000 Evaluations: Average connections for random_seed = 4.6052, std=0.6175216271516327
2023-04-23 13:38:28.168832




COMPILE TIME FOR FLATTENED INDEX (1D) = 27MIN 19 SEC
2023-04-23 08:58:40.162166
BoardName.RANDOM_SEED
num_extendables =  Traced<ShapedArray(int32[])>with<DynamicJaxprTrace(level=1/0)>

2023-04-23 09:12:51.532586: E external/xla/xla/service/slow_operation_alarm.cc:133] The operation took 10m11.74190574s

********************************
[Compiling module jit_return_solved_board] Very slow compile? If you want to file a bug, run with envvar XLA_FLAGS=--xla_dump_to=/tmp/foo and attach the results.
********************************

10000 Evaluations: Average connections for random_seed = 4.6052, std=0.6175216271516327
2023-04-23 09:25:59.110785
        
        
        
COMPILE TIME FOR NESTED ROW/COL FORI_LOOPS = 24MIN 30 SEC
2023-04-23 09:39:07.191526
BoardName.RANDOM_SEED
num_extendables =  Traced<ShapedArray(int32[])>with<DynamicJaxprTrace(level=1/0)>

2023-04-23 09:44:25.666860: E external/xla/xla/service/slow_operation_alarm.cc:65] 
********************************
[Compiling module jit_return_solved_board] Very slow compile? If you want to file a bug, run with envvar XLA_FLAGS=--xla_dump_to=/tmp/foo and attach the results.
********************************
2023-04-23 09:51:10.218737: E external/xla/xla/service/slow_operation_alarm.cc:133] The operation took 8m44.552012049s

********************************
[Compiling module jit_return_solved_board] Very slow compile? If you want to file a bug, run with envvar XLA_FLAGS=--xla_dump_to=/tmp/foo and attach the results.
********************************

10000 Evaluations: Average connections for random_seed = 4.6052, std=0.6175216271516327
2023-04-23 10:03:37.019969


        
COMPILE TIME FOR NESTED FOR LOOPS = 25MIN 24 SEC
2023-04-23 11:42:12.463330
BoardName.RANDOM_SEED
num_extendables =  Traced<ShapedArray(int32[])>with<DynamicJaxprTrace(level=1/0)>

2023-04-23 11:47:36.265151: E external/xla/xla/service/slow_operation_alarm.cc:65] 
********************************
[Compiling module jit_return_solved_board] Very slow compile? If you want to file a bug, run with envvar XLA_FLAGS=--xla_dump_to=/tmp/foo and attach the results.
********************************
2023-04-23 11:54:48.037782: E external/xla/xla/service/slow_operation_alarm.cc:133] The operation took 9m11.772777672s

********************************
[Compiling module jit_return_solved_board] Very slow compile? If you want to file a bug, run with envvar XLA_FLAGS=--xla_dump_to=/tmp/foo and attach the results.
********************************

10000 Evaluations: Average connections for random_seed = 4.6052, std=0.6175216271516327
2023-04-23 12:07:36.375316


COMPARE RESULTS OF ONE-SIDED VS TWO-SIDED EXTENSIONS
TWO-SIDED = 13min 26sec
2023-04-21 14:22:49.151079
BoardName.RANDOM_SEED
10000 Evaluations: Average connections for random_seed = 4.4362, std=0.717446555500826
2023-04-21 14:36:15.766443

ONE-SIDED = 14min 43sec
2023-04-21 15:06:54.433743
BoardName.RANDOM_SEED
10000 Evaluations: Average connections for random_seed = 4.5804, std=0.6371309441551242
2023-04-21 15:21:37.132363

TWO-SIDED EXTENSION IS A LITTLE MORE DIFFICULT AND SLIGHTLY FASTER